In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('day6_data1.csv')
data

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0
...,...,...,...,...
6492,11.2,1.6,3.27,1.0
6493,9.6,8.0,3.15,1.0
6494,9.4,1.2,2.99,1.0
6495,12.8,1.1,3.34,1.0


In [2]:
X = data[['alcohol','sugar','pH']].to_numpy()
Y = data['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
t_x, tt_x, t_y, tt_y = train_test_split(X, Y, random_state=42, test_size=0.2)

In [4]:
s_t_x, v_t_x, s_t_y, v_t_y = train_test_split(t_x, t_y, random_state=42, test_size=0.2)

In [5]:
print(f'학습data:{s_t_x.shape}\n테스트data{v_t_x.shape}\n검증data:{tt_x.shape}')

학습data:(4157, 3)
테스트data(1040, 3)
검증data:(1300, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(s_t_x, s_t_y)
dt.score(s_t_x, s_t_y),dt.score(v_t_x,v_t_y)

(0.9971133028626413, 0.864423076923077)

In [7]:
from sklearn.model_selection import cross_validate
sc = cross_validate(dt, t_x, t_y)
sc

{'fit_time': array([0.00601387, 0.00499487, 0.00499511, 0.0050118 , 0.0040009 ]),
 'score_time': array([0.00100636, 0.        , 0.00100207, 0.        , 0.00100017]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [8]:
np.mean(sc['test_score'])

0.855300214703487

In [9]:
from sklearn.model_selection import StratifiedKFold # 기본값 5폴드 교차검증.
sc1 = cross_validate(dt, t_x, t_y, cv = StratifiedKFold()) 
np.mean(sc1['test_score'])

0.855300214703487

In [10]:
sc_ck = StratifiedKFold(n_splits = 10, shuffle = True, random_state=42) # 10폴드 교차검증.
sc2 = cross_validate(dt, t_x, t_y, cv = sc_ck)
np.mean(sc2['test_score'])

0.8574181117533719

In [11]:
from sklearn.model_selection import GridSearchCV
#DecisionTreeClassifier(min_impurity_decrease=)
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [12]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(t_x, t_y)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [13]:
dt = gs.best_estimator_
dt.score(t_x, t_y), dt.score(tt_x, tt_y)

(0.9615162593804117, 0.8653846153846154)

In [14]:
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [15]:
gs.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [16]:
gs.cv_results_

{'mean_fit_time': array([0.00780072, 0.00639915, 0.00479898, 0.00559969, 0.00580091]),
 'std_fit_time': array([0.00040002, 0.00049105, 0.00074766, 0.00102197, 0.0003998 ]),
 'mean_score_time': array([0.00160112, 0.00160093, 0.00100107, 0.00080085, 0.00120044]),
 'std_score_time': array([4.90388955e-04, 4.90330247e-04, 5.30983387e-07, 4.00424809e-04,
        4.00424071e-04]),
 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_impurity_decrease': 0.0001},
  {'min_impurity_decrease': 0.0002},
  {'min_impurity_decrease': 0.0003},
  {'min_impurity_decrease': 0.0004},
  {'min_impurity_decrease': 0.0005}],
 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]),
 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]),
 'split2_test_score': array([0.8825794 , 0.8

In [17]:
i = np.argmax(gs.cv_results_['mean_test_score'])

In [18]:
gs.cv_results_['params'][i]

{'min_impurity_decrease': 0.0001}

In [19]:
params = {'max_depth':range(5,20,1),
         'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
         'min_samples_split':range(2,100,10)
        }
gs1 = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs1.fit(t_x, t_y)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [20]:
gs1.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [21]:
np.max(gs1.cv_results_['mean_test_score'])

0.8683865773302731

랜덤서치

In [22]:
from scipy.stats import uniform, randint
d = randint(0,10)
d.rvs(5)

array([5, 0, 8, 6, 7])

In [23]:
np.unique(d.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([104,  97, 112,  90, 122,  97,  99,  96,  81, 102], dtype=int64))

In [24]:
d1 = uniform(0,1)
d1.rvs(5)
#np.unique(d1.rvs(1000), return_counts=True)

array([0.70969111, 0.61460917, 0.92727786, 0.98675084, 0.04467224])

In [28]:
params = {'max_depth':randint(20,50),
         'min_impurity_decrease':uniform(0.0001,0.001),
         'min_samples_split':randint(2,25),
         'min_samples_leaf':randint(1,25)
        }

In [29]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,n_iter=100,n_jobs=-1,random_state=42)
rs.fit(t_x, t_y)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000014A583851C0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000014A5AC8E6A0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000014A5AC8E610>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000014A59670FD0>},
                   random_state=42)

In [30]:
rs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}